In [1]:
from models import SonNet,DynamicSonNet
import pickle
import torch, time
from torch import optim
import torch.nn.functional as F


def accuracy(output, labels):
    '''
    get accuracy
    :param output:
    :param labels:
    :return:
    '''
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum().item()
    return correct / len(labels)


In [2]:
# load data
datas = []
for i in range(1000):
    path = f'data/SBM/{i}_uncopynode.pkl'
    with open(path, 'rb') as f:
        datas.append(pickle.load(f).to('cuda:0'))
torch.cuda.manual_seed(0)

In [35]:
# update code
# supermask = [5, 2, 1+12*1, 4+12*2, 5+12*3, (5+12*4)*0, (5+12*5)*0, 5]  # random sure
# supermask = [3, 4, 5+12*1, 9+12*2, 2, 12+12*4, 0, 2] # agcns sure
# supermask = [5, 10, 6+12*1, 1+12*2, 2, 0, 0, 5]  # graphnas sure
# supermask = [3, 1, 1+12*1, 10+12*2, 2+12*2, 0, 0, 4]  # darts sure
# supermask = [4, 1, 1+12*1, 1+12*2, 10, 2, 0, 4]  # fednas sure
# supermask = [5, 8, 3+12*1, 1+12*2, 10+12*1, 0, 0, 4] # rl
supermask = [5, 4, 4+12*1, 4+12*2, 4+12*3, (4+12*4)*1, (4+12*5)*1, 5]  # try
with open('tmp.pkl', 'wb') as f:
    pickle.dump(supermask, f)


In [36]:
# train model
model = SonNet(6,6).to('cuda:0')
model.train()
optimizer = optim.Adam(
    model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5)
st_time = time.time()
for epoch in range(50):
    losses = 0
    train_accu = 0
    val_accu = 0
    test_accu = 0
    best_val_accu = 0
    for iter, data in enumerate(datas):
        preds = model(data.x, data.edge_index)
        labels = data.y[data.train_mask]
        loss = F.cross_entropy(
            preds[data.train_mask], data.y[data.train_mask], weight=data.weight)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses += loss.item()
        train_accu += accuracy(preds[data.train_mask],
                                data.y[data.train_mask])
        val_accu += accuracy(preds[data.val_mask],
                                data.y[data.val_mask])
        test_accu += accuracy(preds[data.test_mask],
                                data.y[data.test_mask])
    if val_accu > best_val_accu:
        best_val_accu = val_accu
        torch.save(model.state_dict(), 'sbm_model.pth')
        print('Iter{}, loss:{},train_acc:{},val_acc:{},test_acc:{},use_time:{}'.format(
            epoch+1, losses/(iter+1), train_accu/(iter+1), val_accu/(iter+1), test_accu/(iter+1), round(time.time()-st_time)))
    scheduler.step(losses/(iter+1))

test_accu = 0
model.load_state_dict(torch.load('sbm_model.pth'))
model.eval()
for iter, data in enumerate(datas):
    preds = model(data.x, data.edge_index)
    test_accu += accuracy(preds[data.test_mask],
                            data.y[data.test_mask])*len(data.y[data.test_mask])/116820
print(f'test accu:{test_accu}')


tensor([0., 0., 0., 0., 0., 0., 1.])
Iter1, loss:1.6383319321870804,train_acc:0.3199542666654383,val_acc:0.32138607251551937,test_acc:0.3236557709312128,use_time:15


KeyboardInterrupt: 